Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [2]:
#Ваш код здесь
males = df.loc[df['gender'] == 'Male', 'gender'].count()
females = df.loc[df['gender'] == 'Female', 'gender'].count()
male_to_female_ratio = males / females
male_to_female_ratio

1.0192087155963303

##### 2. Какое количество уникальных значений у поля InternetService?

In [3]:
#Ваш код здесь
df['InternetService'].value_counts().count()

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [4]:
#Ваш код здесь
def clear_series(series: pd.Series, fillna_value=0):
    series = series.copy()
    series = series.replace(r'\s+', np.nan, regex=True)
    if not fillna_value is None:
        series.fillna(fillna_value, inplace=True)
    series = pd.to_numeric(series)
    return series
    
def get_stat(series: pd.Series, stat: str, clear=False, inplace=True):
    if not inplace:
        series = series.copy()
        
    if clear:
        series = clear_series(series)
        
    stat = stat.lower()
    if stat == 'median':
        return series.median()
    elif stat == 'mean':
        return series.mean()
    elif stat == 'std':
        return series.std()

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

In [5]:
df["TotalCharges"].dtype

dtype('O')

In [6]:
df["TotalCharges"].value_counts()

20.2      11
          11
19.75      9
20.05      8
19.9       8
          ..
86.35      1
294.95     1
4014       1
408.5      1
4931.8     1
Name: TotalCharges, Length: 6531, dtype: int64

тип данных object вместо ожидаемого float
также в данных есть значения ' '

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [7]:
#Ваш код здесь
def to_binary(series: pd.Series, schema: dict=None):
    series = series.copy()
    if schema is None:
        schema = series.value_counts()
        schema = pd.Series([i for i in range(series.shape[0])], 
                           index=series.index)
        schema = schema.to_dict()
    for key, value in schema.items():
        series.loc[series == key] = value
    return series

df['PhoneService'] = to_binary(df['PhoneService'], schema={'Yes': 1, 'No': 0})
df['PhoneService'] 

0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: object

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [8]:
#Ваш код здесь
stats = ('median', 'mean', 'std')
for stat in stats:
    print(f'{stat}: {get_stat(df["TotalCharges"], stat, clear=True)}')

median: 1394.55
mean: 2279.734303563826
std: 2266.7944696890195


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [9]:
#Ваш код здесь
df['Churn'] = to_binary(df['Churn'], schema={'Yes': 1, 'No': 0})
df['Churn'] 

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: object

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [10]:
#Ваш код здесь
features = ['StreamingMovies', 'StreamingTV', 'TechSupport']
for feature in features:
    df[feature] = to_binary(df[feature], 
                            schema={
                                'Yes': 1, 
                                'No': 0, 
                                'No internet service': 0
                            })

##### 8. Заполните пропуски в поле PhoneService значением 0

In [11]:
#Ваш код здесь
df['PhoneService'].fillna(0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [12]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь
df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.5,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [13]:
# где-то есть ''-значения (задание 13 выдавало ошибку)
numerical = ['tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
for feature in numerical:
    df[feature] = clear_series(df[feature])

In [14]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь
x_train, x_test, y_train, y_test = train_test_split(
    df[features], df[target], test_size=0.3, random_state=42
)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]    
    
gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [16]:
#Ваш код здесь
gender.fit(x_train, y_train)

Pipeline(steps=[('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))])

##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [17]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [18]:
#Ваш код здесь
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [19]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [20]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(x_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [22]:
#Ваш код здесь
y_pred = pipeline.predict_proba(x_test)
y_pred

array([[0.29, 0.71],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.93, 0.07],
       [0.23, 0.77],
       [0.94, 0.06]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [23]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь
y_pred = pd.DataFrame(y_pred)
print(f'roc_auc_score: {roc_auc_score(y_test, y_pred[1])}')
print(f'log_loss: {log_loss(y_test, y_pred[1])}')

roc_auc_score: 0.7920778685648177
log_loss: 0.7240560773970856


### Сохраним наш пайплайн

In [24]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)